In [1]:
import io
import re
import string
import tqdm

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [3]:
sentence = 'The wide road shimmered in the hot sun'
tokens = list(sentence.lower().split())
print(len(tokens))

8


In [4]:
vocab, index = {}, 1
vocab['<pad>'] = 0
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [5]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [6]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


In [7]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
  example_sequence,
  vocabulary_size=vocab_size,
  window_size=window_size,
  negative_samples=0
)
print(len(positive_skip_grams))

26


In [8]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(2, 3): (wide, road)
(4, 3): (shimmered, road)
(3, 2): (road, wide)
(4, 2): (shimmered, wide)
(6, 7): (hot, sun)


In [21]:
target_word, context_word = positive_skip_grams[0]

num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype='int64'), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
  true_classes=context_class,
  num_true=1,
  num_sampled=num_ns,
  unique=True,
  range_max=vocab_size,
  seed=SEED,
  name='negative_sampling'
)

print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([0 7 4 6], shape=(4,), dtype=int64)
['<pad>', 'sun', 'shimmered', 'hot']


In [22]:
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

context = tf.concat([context_class, negative_sampling_candidates], 0)

label = tf.constant([1] + [0] * num_ns, dtype='int64')

target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

In [23]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 2
target_word     : wide
context_indices : [3 0 7 4 6]
context_words   : ['road', '<pad>', 'sun', 'shimmered', 'hot']
label           : [1 0 0 0 0]


In [24]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : tf.Tensor(2, shape=(), dtype=int32)
context : tf.Tensor([3 0 7 4 6], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [25]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [44]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  targets, contexts, labels = [], [], []
  
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)
  
  for sequence in tqdm.tqdm(sequences):
    
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      sequence,
      vocabulary_size=vocab_size,
      sampling_table=sampling_table,
      window_size=window_size,
      negative_samples=0
    )
    
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
        tf.constant([context_word], dtype='int64'), 1
      )
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
        true_classes=context_class,
        num_true=1,
        num_sampled=num_ns,
        unique=True,
        range_max=vocab_size,
        seed=SEED,
        name='negative_sampling'
      )
      
      negative_sampling_candidates = tf.expand_dims(
        negative_sampling_candidates,
        1
      )
      
      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0] * num_ns, dtype='int64')
      
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)
      
  return targets, contexts, labels

In [27]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [28]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
  
for line in lines[:20]:
  print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [29]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [30]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(
    lowercase,
    '[%s]' % re.escape(string.punctuation), ''
  )

vocab_size = 4096
sequence_length = 10

vectorize_layer = layers.TextVectorization(
  standardize=custom_standardization,
  max_tokens=vocab_size,
  output_mode='int',
  output_sequence_length=sequence_length
)

In [31]:
vectorize_layer.adapt(text_ds.batch(1024))

2022-07-13 22:59:28.979983: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [32]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [33]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [34]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


In [35]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [46]:
targets, contexts, labels = generate_training_data(
  sequences=sequences,
  window_size=2,
  num_ns=4,
  vocab_size=vocab_size,
  seed=SEED
)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

100%|██████████████████████████████████| 32777/32777 [00:03<00:00, 10049.14it/s]




targets.shape: (65444,)
contexts.shape: (65444, 5)
labels.shape: (65444, 5)


In [47]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000

dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [48]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [49]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(
      vocab_size,
      embedding_dim,
      input_length=1,
      name='w2v_embedding'
    )
    self.context_embedding = layers.Embedding(
      vocab_size,
      embedding_dim,
      input_length=num_ns + 1
    )
    
  def call(self, pair):
    target, context = pair
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    
    word_emb = self.target_embedding(target)
    context_emb = self.context_embedding(context)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    return dots

In [50]:
def custom_loss(x_logits, y_true):
  return tf.nn.sigmoid_cross_entropy_with_logits(
    logits=x_logts,
    labels=y_true,
  )

In [51]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(
  optimizer='adam',
  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['accuracy']
)

In [52]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir='logs')

In [53]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20
63/63 [==============================] - 0s 3ms/step - loss: 1.6082 - accuracy: 0.2342
Epoch 2/20
63/63 [==============================] - 0s 4ms/step - loss: 1.5886 - accuracy: 0.5538
Epoch 3/20
63/63 [==============================] - 0s 5ms/step - loss: 1.5407 - accuracy: 0.5975
Epoch 4/20
63/63 [==============================] - 0s 5ms/step - loss: 1.4583 - accuracy: 0.5713
Epoch 5/20
63/63 [==============================] - 0s 5ms/step - loss: 1.3605 - accuracy: 0.5774
Epoch 6/20
63/63 [==============================] - 0s 5ms/step - loss: 1.2639 - accuracy: 0.6052
Epoch 7/20
63/63 [==============================] - 0s 5ms/step - loss: 1.1737 - accuracy: 0.6401
Epoch 8/20
63/63 [==============================] - 0s 5ms/step - loss: 1.0897 - accuracy: 0.6750
Epoch 9/20
63/63 [==============================] - 0s 5ms/step - loss: 1.0115 - accuracy: 0.7081
Epoch 10/20
63/63 [==============================] - 0s 4ms/step - loss: 0.9387 - accuracy: 0.7398
Epoch 11/20
63/63 [

In [56]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [57]:
%tensorboard --logdir logs

In [58]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [59]:
out_v = io.open('word2vec_vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('word2vec_metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()